In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
SEED = 8153
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device=device_by_name("1060")
torch.cuda.set_device(device)
#device = "cpu"
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet161-Copy2-2 results')

In [3]:
def get_submission(test_df,pred):
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':torch.sigmoid(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':torch.sigmoid(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':torch.sigmoid(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':torch.sigmoid(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':torch.sigmoid(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':torch.sigmoid(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [4]:
def get_submission_ids(image_ids,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [ ]:
train_df = pd.read_csv(data_dir+'train.csv')
train_df.shape
train_df=train_df[~train_df.PatientID.isin(bad_images)].reset_index(drop=True)
train_df=train_df.drop_duplicates().reset_index(drop=True)
train_df.shape
train_df.head()

(674510, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


In [ ]:
test_df = pd.read_csv(data_dir+'test.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


In [ ]:
split_sid = train_df.PID.unique()
splits=list(KFold(n_splits=3,shuffle=True, random_state=SEED).split(split_sid))


In [ ]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [ ]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [ ]:
#features=(features-features.mean())/features.std()

In [ ]:
class ResModel(nn.Module):
    def __init__(self,in_size):
        super(ResModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (5,2208), padding=(2,0))
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 6, 1)

        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.relu0(x)
        out = self.conv1d1(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [ ]:
class ResModelPool(nn.Module):
    def __init__(self,in_size):
        super(ResModelPool, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size),stride=(1,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
#        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x)
        x=F.max_pool2d(x,kernel_size=(1,x.shape[-1])).squeeze(-1)        
        x0 = self.bn0(x)
#        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [ ]:
multi=3
model_names=['se_resnext101_32x4d','se_resnext101_32x4d','Densenet161_3','Densenet169_3','Densenet201_3','se_resnet101']
types=['features_train_tta','features_train_tta','features_train_tta2','features_train_tta2','features_train_tta','features_train_tta']
versions=['classifier_splits','new_splits','classifier_splits','classifier_splits','classifier_splits','classifier_splits']
num_splits=[3,5,3,3,3,3]
seeds=[8153,432,8153,8153,8153,8153]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        split_sid = train_df.PID.unique()
        splits=list(KFold(n_splits=n,shuffle=True, random_state=SEED).split(split_sid))

        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//4,4,-1)
        split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =3)

        win_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


se_resnext101_32x4d classifier_splits features_train_tta 0


<All keys matched successfully>

tensor(224270)

se_resnext101_32x4d classifier_splits features_train_tta 1


<All keys matched successfully>

...

<All keys matched successfully>

tensor(134734)

se_resnext101_32x4d new_splits features_train_tta 1


<All keys matched successfully>

tensor(131494)

In [ ]:
multi=3
model_names=['se_resnext101_32x4d','se_resnext101_32x4d','Densenet161_3','Densenet169_3','Densenet201_3','se_resnet101']
types=['features_test_tta','features_test','features_test2','features_test2','features_test','features_test']
versions=['classifier_splits','new_splits','classifier_splits','classifier_splits','classifier_splits','classifier_splits']
num_splits=[3,5,3,3,3,3]
seeds=[8153,432,8153,8153,8153,8153]
for model_name,type_,version_,n,SEED in zip(model_names,types,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//4,4,-1)
        split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =4)

        win_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensamble',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


In [ ]:
multi=3
for num_split in tqdm_notebook(range(3)):
    pred_list=[]
    model_name,version = 'se_resnet101' , 'classifier_splits'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features=features.reshape(features.shape[0]//4,4,-1)
    split_all =  train_df.SeriesI.unique()
    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_{}'.format(multi)

    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    
    valid_dataset=FullHeadDataset(train_df,
                                  split_all,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  multi =3)
    
    win_dataset=FullHeadDataset(train_df,
                                  split_all,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                   target_columns=hemorrhage_types)
    win_list=[]
    dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
    for _,win in tqdm_notebook(dl):
        win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
    wins = torch.cat(win_list,0).sum(1)>=0
    wins.sum()
    for i in tqdm_notebook(range(32),leave=False):
        pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
        pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'all_pred',num_split),'wb')
    pickle.dump(pred_list,pickle_file,protocol=4)
    pickle_file.close()


<All keys matched successfully>

tensor(674252)

<All keys matched successfully>

tensor(674252)

<All keys matched successfully>

tensor(674252)

In [35]:
OOF_ids={}
SEED=432
n=5
split_sid = train_df.PID.unique()
splits=list(KFold(n_splits=n,shuffle=True, random_state=SEED).split(split_sid))

for i in range( n):
    images_id_list=[]
    
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=train_df.PatientID.values
    ref_arr=train_df.SeriesI.values
    order_arr=train_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    image_ids.shape
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_5.pkl','wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


(134734,)

(131494,)

(136434,)

(135582,)

(136008,)

In [35]:
OOF_ids={}
SEED=8153
n=3
split_sid = train_df.PID.unique()
splits=list(KFold(n_splits=n,shuffle=True, random_state=SEED).split(split_sid))

for i in range( n):
    images_id_list=[]
    
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=train_df.PatientID.values
    ref_arr=train_df.SeriesI.values
    order_arr=train_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    image_ids.shape
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_5.pkl','wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


(134734,)

(131494,)

(136434,)

(135582,)

(136008,)

In [21]:
images_id_list=[]
split_all =  train_df.SeriesI.unique()
image_arr=train_df.PatientID.values
ref_arr=train_df.SeriesI.values
order_arr=train_df.ImagePositionZ.values
for s in tqdm_notebook(split_all):
    head_idx = np.where(ref_arr==s)[0]
    sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
    images_id_list.append(image_arr[sorted_head_idx])
image_ids=np.concatenate(images_id_list)
image_ids.shape



(674252,)

In [23]:
pickle_file=open(outputs_dir+'all_image_ids.pkl','wb')
pickle.dump(image_ids,pickle_file,protocol=4)
pickle_file.close()


/home/reina/anaconda3/envs/RSNA/lib/python3.6/site-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='/media/hd/notebooks/data/RSNA/outputs/all_image_ids.pkl'>
  """Entry point for launching an IPython kernel.


In [14]:
images_id_list=[]
dummeys=[]
image_arr=test_df.PatientID.values
ref_arr=test_df.SeriesI.values
order_arr=test_df.ImagePositionZ.values
for s in tqdm_notebook(test_df.SeriesI.unique()):
    dumm=np.zeros(60)
    head_idx = np.where(ref_arr==s)[0]
    sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
    images_id_list.append(image_arr[sorted_head_idx])
    dumm[0:head_idx.shape[0]]=1
    dummeys.append(dumm)
image_ids=np.concatenate(images_id_list)
select=np.concatenate(dummeys)==1

In [23]:
pickle_file=open(outputs_dir+'ensemble_test_image_ids.pkl','wb')
pickle.dump(image_ids,pickle_file,protocol=4)
pickle_file.close()


In [18]:
for num_split in tqdm_notebook(range(3)):
    pred_list=[]
    model_name,version = 'Densenet161_3' , 'classifier_splits'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test2',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features=features.reshape(features.shape[0]//8,8,-1)

    model=ResModel(features.shape[-1])
    version=version+'_fullhead_resmodel'

    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    test_dataset=train_dataset=FullHeadDataset(test_df,
                                  test_df.SeriesI.unique(),
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ')
    for i in tqdm_notebook(range(32),leave=False):
        p=model_run(model,test_dataset,do_apex=False,batch_size=128)
        pred_list.append(p.reshape(p.shape[0]*p.shape[1],-1)[select])
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensamble',num_split),'wb')
    pickle.dump(pred_list,pickle_file,protocol=4)
    pickle_file.close()



<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>